# My First AI Agent

<div>
<center><img src="./images/time_agent.png" width="600"/></center>
</div>

In this exercise, you will use [**LangGraph**](https://www.langchain.com/langgraph) and its [`create_react_agent`](https://langchain-ai.github.io/langgraph/reference/prebuilt/) utility function to build a ReAct AI agent with a simple yet specific task: **providing the current date and time in a specific location**. This agent will respond to questions such as:

**"What time is it in Italy?"**

To achieve this, the agent will follow these steps:
1. Understand the task the user wants it to perform.
2. Map the user-provided location to its corresponding timezone using its internal knowledge.
3. Use a tool to retrieve the current date and time for that timezone.
4. Generate a response based on the tool's output to answer the user’s query.

This exercise will give you hands-on experience with **LangGraph** and the process of building AI agents equipped with reasoning and tool-using capabilities.

Let’s get started!

## 0. Preparing the Environment

Before building the AI agent, we need to set up our environment and import the necessary functions and libraries. In this exercise, we will use:

- [`load_dotenv`](https://pypi.org/project/python-dotenv/): To load environment variables (e.g., your OpenAI API key).
- [`ChatOpenAI`](https://python.langchain.com/api_reference/openai/chat_models/langchain_openai.chat_models.base.ChatOpenAI.html): A language model interface for interacting with OpenAI's GPT models.
- [`create_react_agent`](https://langchain-ai.github.io/langgraph/reference/prebuilt/): A utility from **LangGraph** for building ReAct AI agents.
- [`tool`](https://python.langchain.com/api_reference/core/tools/langchain_core.tools.convert.tool.html): A decorator to define tools that the agent can use during execution.
- `print_stream`: A utility to print the agent’s responses as they stream in.

In [ ]:
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from utils.stream import print_stream

load_dotenv(override=True)

## 1. Initializing the Large Language Model (LLM)

The first step to create a new AI Agent with LangGraph is to initialize the OpenAI language model that our agent will use to process and respond to queries. When specifying the model, it's a good practice to:

1. **Specify the full model name**: To ensure compatibility and stability (e.g., `gpt-4o-mini-2024-07-18`).
2. **Start with a low temperature value**: A temperature of `0` ensures deterministic responses, making the agent focus on precision. You can increase the temperature later for more creative responses.

For this exercise, we will use the latest version of the `gpt-4o-mini` model (`gpt-4o-mini-2024-07-18`) with a temperature of `0`, but you can experiment with other models (the full list is available [here](https://platform.openai.com/docs/models#current-model-aliases)), temperatures, and other parameters to see how they affect the agent's behavior.

In [ ]:
MODEL_NAME = "openai:gpt-4o-mini-2024-07-18"
MODEL_TEMPERATURE = 0

llm = init_chat_model(model=MODEL_NAME, temperature=MODEL_TEMPERATURE)

You can learn more about ChatOpenAI in the official langchain documentation: https://python.langchain.com/docs/integrations/chat/openai/

## 2. Test the LLM

Before equipping our agent with tools, let’s verify that the underlying language model lacks access to real-time information such as the current date and time. This is expected behavior because the LLM operates on static knowledge and does not have tools or APIs to fetch live data on its own.

We’ll confirm this by asking the LLM the following question:  

**"What time is it in Italy?"**

Let’s run the code and observe the response.

In [ ]:
# You can learn more about langchain messages and roles in the official documentation: https://python.langchain.com/docs/concepts/messages/
messages = [
    ("user", "What time is it in Italy?")
]

response = llm.invoke(messages)

print("Assistant:", response.content)

## 3. Defining the Tools

Now that we have seen that the LLM cannot access real-time information on its own, we can give the AI agent a **tool** to fill this gap. Tools are external functions that the agent can use to assist in answering user queries.

### Tool Requirements:

For this exercise, we are building a tool to retrieve the current date and time for a given timezone. Here's what the tool should do:
1. Accept an input, specifically a timezone.
2. Determine the current date and time for that timezone.
3. Return the result to the agent.
4. Manage any errors that may occur during the process.

You can define your own implementation of the tool (recomended) or use the provided one.

To learn more about tools in LangGraph, you can refer to the official documentation: https://python.langchain.com/docs/concepts/tools/

In [ ]:
from tools.date_and_time import prebuilt_get_current_datetime

@tool
def get_current_datetime(timezone: str) -> str:
    """TODO: Use this space to describe this tool to the agent.
    Try to answer the following questions:
        - What does this tool do?
        - What is the input?
        - How should the input be formatted?
    You can also provide examples of how to use this tool to help the agent understand it better.
    """
    #TODO Implement the function logic here or use the prebuilt version
    # return prebuilt_get_current_datetime(timezone)

Test the tool by calling it with a sample timezone, such as `"Europe/Rome"`, and observe the output.

In [ ]:
get_current_datetime.invoke("Europe/Rome")

## 4. Crafting the Agent System Prompt

Before integrating tools and building our agent, we need to define the **system prompt**. The system prompt acts as the foundation for the agent's behavior by clearly outlining its role, purpose, instructions, and limitations.

Crafting the perfect system prompt is an iterative process, so you can start with a simple version and refine it as you test the agent's performance.

### Prompt Engineering Techniques:
Here are some techniques to keep in mind when defining the agent's system prompt:
1. **Role Prompting**: Clearly define the role of the agent (e.g., "You are D&T Bot, an agent specialized in providing the current date and time for different locations.").
2. [**Few-Shot Prompting**](https://www.promptingguide.ai/techniques/fewshot) (Optional): Provide a few examples of user questions and the appropriate responses to guide the agent further.

You can learn more about prompt engineering in the prompting engineering guide: [https://www.promptingguide.ai/](https://www.promptingguide.ai/)

### Your Task:
Define a system prompt for your agent. The agent should:
- Clearly convey its **role and purpose**.
- Provide the **current date and time** for a specific location when asked.
- Ask the user to specify a location if it is not provided.
- Politely decline to answer any queries unrelated to date and time.

### Things to Consider:
- **No Need to Describe Tools**: You do not need to specify the tools or their behaviors in the system prompt. The agent will already know how to use them based on the descriptions you provided earlier.
- **Be Creative**: You can inject some personality into your prompt if desired to give your agent a unique tone (e.g., friendly, formal, or professional).

In [ ]:
DATE_AND_TIME_AGENT_PROMPT = """"#TODO: Define your agent system prompt here."""

## 5. Building the AI Agent

Now that we have all the core components ready:
1. The **language model (LLM)** for processing queries.
2. A **tool** for retrieving the current date and time.
3. A well-crafted **system prompt** to guide the agent's behavior.

We’re ready to bring everything together into a fully functional AI agent. 

### How It Works:
We’ll use the **`create_react_agent`** function from LangGraph. This utility embeds the logic required to create a basic **ReAct (Reason + Act)** agent, allowing it to:
- Reason about the user’s query.
- Decide whether and how to use the provided tools.
- Generate meaningful responses, respecting the constraints of the system prompt.

### Key Parameters:
- **`model`**: The LLM we defined earlier.
- **`tools`**: A list of tools (e.g., the `get_current_datetime` tool).
- **`prompt`**: The system prompt crafted to guide the agent.

You can learn more about the `create_react_agent` function in the official LangGraph documentation: https://langchain-ai.github.io/langgraph/reference/prebuilt/#langgraph.prebuilt.chat_agent_executor.create_react_agent

In [ ]:
graph = create_react_agent(model=llm, tools=[get_current_datetime], prompt=DATE_AND_TIME_AGENT_PROMPT)

The **ReAct agent** we just created isn't just a simple function; it's a structured graph that represents how the agent reasons, acts, and responds to user input. LangGraph allows us to visualize this graph, which helps us better understand:
- How the agent processes inputs.
- How its decision-making workflow is structured.

Let’s display the graph!

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

## 6. Testing the AI Agent

Now that the agent is fully built, it's time to test it! By sending a query to the agent, we can examine how it:
1. Processes the input question.
2. Reasonably decides if and how to use tools like `get_current_datetime`.
3. Produces an accurate and user-friendly response.

### Testing Query:
Let’s try the same query we used to test the LLM:

**"What time is it in Italy?"**

To verify that the agent is now capable of providing the current date and time for a specific location.

In [ ]:
question = """What time is it in Italy?"""

inputs = {"messages": [("user", question)]}

print_stream(graph.stream(inputs, stream_mode="values"))

Let's try a more complex query:

**"What time is it in Italy and California?"**

This query is intentionally a little tricky so we can observe how the agent handles both locations and whether it uses tools appropriately.

In [ ]:
question = """What time is it in Italy and California?"""

inputs = {"messages": [("user", question)]}

print_stream(graph.stream(inputs, stream_mode="values"))

### Iterative Testing:

Now that your agent is up and running, it’s time to test it with your own queries! This hands-on testing allows you to evaluate:
1. **How well the agent responds to expected queries** (e.g., asking the time in different locations).
2. **Whether it handles unexpected or ambiguous queries gracefully** (e.g., if no location is provided or irrelevant questions are asked).
3. **How effectively it uses the tools** to fulfill its purpose.

### Iterative Process:

Crafting the perfect agent is an **iterative process**. Use these trials to identify areas of improvement and adjust the agent as needed. Consider modifying the following aspects:
- **System Prompt**: Adjust the wording to clarify the agent’s purpose, add reasoning steps, or inject personality.
- **Tool Definition**: Rename tools, change their parameters, or improve their descriptions for clarity and better integration.
- **Model Settings**: Experiment with model temperature to balance determinism (`temperature=0`) and creativity (higher values).
- **Tool Limitations**: For example, add error handling if an invalid timezone is passed.

### Tips for Effective Refinement:
- If your agent isn’t responding as expected, start by reviewing the reasoning traces and the tool execution steps.
- Test with a wider variety of queries to account for edge cases.
- Iterate gradually, making small adjustments to prompts, tools, or configurations one step at a time.

Remember, building an AI agent is like sculpting—it evolves over time with refinement and experimentation.

In [ ]:
question = """<YOUR QUESTION HERE>"""

inputs = {"messages": [("user", question)]}

print_stream(graph.stream(inputs, stream_mode="values"))

## 7. Observing the Agent's Reasoning

For educational purposes, it can be helpful to make a ReAct AI agent **explicitly explain its reasoning steps**. This allows us to:
1. **Understand how the agent thinks**: By viewing its intermediate reasoning steps, we can better grasp how the agent interprets the question and makes decisions.
2. **Debug issues interactively**: If the agent doesn’t perform as expected, we can identify exactly where its reasoning or tool usage went wrong.
3. **Reinforce Prompt Engineering**: Small changes in the system prompt can significantly affect the agent's behavior. Testing these changes provides insight into how to fine-tune prompts for specific workflows.

To force the agent to 'think out loud', we can tell it to do so using a specific system prompt.

In [ ]:
EXPLICIT_DATE_AND_TIME_REACT_AGENT_PROMPT = "You are a ReAct agent. Explicitly state the reasoning before each tool call. #TODO: Define the rest of your agent system prompt here."

In [ ]:
from langchain_core.messages.system import SystemMessage

explicit_graph = create_react_agent(model=llm, tools=[get_current_datetime], prompt=EXPLICIT_DATE_AND_TIME_REACT_AGENT_PROMPT)

In [ ]:
question = """What time is it in Italy and California?"""

inputs = {"messages": [("user", question)]}

print_stream(explicit_graph.stream(inputs, stream_mode="values"))

## 8. Add Short Term Memory to the Agent

Until now, our agent operates in a **stateless mode**, meaning it processes each query independently without any awareness of previous interactions. For example, if you ask the agent:

- *"What is the last thing I asked you?"*

The agent will fail to respond meaningfully because it doesn't retain previous conversation history.

### Why Add Memory?

Adding **memory** enables the agent to:
1. **Support multi-turn conversations**: It will remember context from earlier queries and use that information in future responses.
2. **Improve user experience**: This creates a more natural, conversational flow, where the agent can track and respond to follow-up questions or clarifications.
3. **Enable full conversational applications**: With memory, the agent can handle ongoing interactions without asking repetitive questions or losing track of the dialogue.

You can learn more about memory in LangGraph in the official documentation: https://langchain-ai.github.io/langgraph/how-tos/persistence/

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

# Initialize MemorySaver
memory = MemorySaver()

graph_with_memory = create_react_agent(
    model=llm,
    tools=[get_current_datetime],
    prompt=DATE_AND_TIME_AGENT_PROMPT,
    checkpointer=memory,
)

In [ ]:
from uuid import uuid4

config = {
    "configurable": {
        "thread_id": str(uuid4()) # The thread_id is a unique identifier for each conversation thread.
        }
}

In [ ]:
question = """What time is it in Italy?"""

inputs = {"messages": [("user", question)]}

print_stream(graph_with_memory.stream(inputs,  config, stream_mode="values"))

In [ ]:
question = """What is the last question I asked you?"""

inputs = {"messages": [("user", question)]}

print_stream(graph_with_memory.stream(inputs, config, stream_mode="values"))

## 9. Conclusion

🎉 Congratulations on Completing the exercise! 🎉

Well done! You've successfully built your first AI agent using **LangGraph** and explored its capabilities step by step. 

### What We’ve Accomplished:
1. **Understanding Stateless Assistants**: You saw how an LLM alone cannot provide real-time information without tools or resources.
2. **Defining Tools**: You created a custom tool to retrieve the current date and time for specific time zones using Python and LangGraph's `@tool` decorator.
3. **Crafting a System Prompt**: Using prompt engineering, you wrote clear instructions for the agent to guide its reasoning and behavior.
4. **Building the Agent**: Using `create_react_agent`, you orchestrated the `LLM`, tools, and system prompt into a fully functional ReAct AI agent.
5. **Testing and Refining**: You tested the agent with sample queries, refined its behavior with explicit reasoning, and enabled multi-turn conversations by incorporating **memory**.

### What’s Next?

You’re now ready to move to the **next notebook**! In the second exercise, you’ll deepen your understanding of LangGraph by:
- Exploring more advanced tools and functionalities.
- Building a more complex agent with additional reasoning and decision-making capabilities.

Head over to Notebook "**2 - My Second AI Agent.ipynb**" to continue your journey and build upon the foundational knowledge you've gained here.

🚀 Keep up the amazing work, and let’s make your next agent even smarter!